# Limpieza y Transformación de los Datos

En esta fase, se realizará la gestión de duplicados, columnas o filas irrelevantes, tipificación incorrecta de datos y normalización de datos (mismo idioma, mayúsculas/minúsculas, etc.).

1. En primer lugar, eliminamos las filas irrelevantes, es decir, aquellas que contienen información duplicada o ninguna. Para ello, utilizamos la opción "Quitar Duplicados" del panel de diseño de tabla. Se eliminaron 14 duplicados. No se observaron filas sin información.

2. A continuación, analizamos las columnas irrelevantes. Comprobamos si la columna "Churned" coincidía con la etiqueta "Churned Label". Por ello, creamos una columna adicional para indicar si los datos son correctos o incorrectos si se cumplían las siguientes condiciones:


In [ ]:
=SI(O(Y([@[Churn Label]]="No";[@Churned]=0);Y([@[Churn Label]]="Yes";[@Churned]=1));"OK";"Not OK")

Gracias al filtro de esa columna adicional, observamos que ninguna fila estaba etiquetada como "Not OK", por lo que se pudo eliminar una de las columnas, ya que representaban lo mismo. Decidimos eliminar el campo "Churned" porque queríamos mantener las etiquetas (Sí/No) para el panel final. Para la fase de transformación, consideramos interesante convertir los campos booleanos adicionales en etiquetas Sí/No, si corresponde.

También eliminamos la columna "Phone number", ya que era irrelevante para nuestro análisis.

Observamos algunas inconsistencias en las diferentes categorías del campo "Gender". Dos de ellas están en inglés (Female / Prefer not to tell) y la restante en español (Hombre).

Por lo tanto, decidimos mantener las columnas "Under30" y "Senior" porque pensamos que podrían estar relacionadas con un KPI de nuestro cliente (la alternativa era eliminar ambas y crear nuestros propios grupos de edad a partir del campo "Age"). Sin embargo, primero queríamos validar esos campos comparando sus etiquetas (Yes/No) con el valor real del campo "Age". Por ese motivo, creamos columnas adicionales para indicar OK o Not OK si se cumplen las siguientes condiciones:

In [ ]:
=SI(O(Y([@Age]>=30;[@[Under 30]]="No");Y([@Age]<30;[@[Under 30]]="Yes"));"OK";"Not OK")

=SI(O(Y([@Age]>65;[@Senior]="Yes");Y([@Age]<=65;[@Senior]="No"));"OK";"Not OK")

Los valores de las etiquetas del campo "Under 30" coincidían con los valores de edad del campo "Edad". Sin embargo, algunos valores del campo "Senior" presentaban inconsistencias. En algunos casos, los suscriptores de 65 años se etiquetaron como "Senior" (OK) y otros como "No senior" (Not OK). Decidimos no modificar esa columna sin antes solicitar al cliente que aclare estas inconsistencias.

Nuestro cliente nos confirma que los que tienen 65 años son considerados como Senior. Por lo que modificamos la columna "Senior" para que todos los suscriptores de 65 años se etiqueten como "Yes".



In [ ]:
=SI[@Age]>=65;"Yes","No"

Aprovechando las fórmulas anteriores, también validamos los datos del campo "Number of Customers in Group" para comprobar si sus valores coincidían con las etiquetas "Yes/No" del campo "Group".

In [ ]:
=SI(O(Y([@Group]="No";[@[Number of Customers in Group]]<2);Y([@Group]="Yes";[@[Number of Customers in Group]]>1));"OK";"Not OK")

Todos los valores eran correctos.

Tras preguntar al cliente sobre el campo Customer Ternure (in months), respondió que no era relevante para el análisis, ya que no tenía nada que ver con el tema. Por lo tanto, decidimos eliminarlo.

Otras columnas irrelevantes que se eliminaron fueron: Hobby, Favorite TV Show, Internal Notes.

3. El siguiente paso fue corregir la escritura incorrecta de datos. El campo Monthly Charge contenía algunos valores con el sufijo USD y otros sin él. Aplicamos las siguientes fórmulas anidadas en una nueva columna (Monthly Charge (USD)) para extraer solo los números y convertirlos a datos numéricos:

In [ ]:
=VALOR.NUMERO(EXTRAE([@[Monthly Charge]];1;SI.ERROR(HALLAR(" ";[@[Monthly Charge]]);[@[Monthly Charge]])))

El campo original de Cargo Mensual estaba oculto.

A continuación, creamos una nueva columna de Contact Date, mediante la siguiente fórmula. La columna original la cambiamos de nombre a "Contact Date raw" y la ocultamos.

In [ ]:
=FECHANUMERO([@[Contact Date raw]])

Para finalizar la corrección de los tipos de datos, transformamos los valores del campo "Average Monthly Expenses" a números y aplicamos dos decimales.

4. Una vez definidos correctamente los tipos de datos, intentamos realizar la normalización de datos. Creamos una nueva columna del campo "Unlimited Data Plan" para convertir los valores booleanos en etiquetas de "Yes/No". La columna original la cambiamos de nombre a "Unlimited Data Plan raw" y la ocultamos.

In [ ]:
=SI([@[Unlimited Data Plan raw]]=0;"No";"Yes")

Además, normalizamos las categorías del campo "Gender" para asegurarnos de que todas estuvieran escritas en inglés. Para ello, cambiamos el nombre del campo a "Genger raw" y creamos una nueva columna llamada "Gender", donde aplicamos la siguiente fórmula condicional:

In [ ]:
=SI([@[Gender raw]]="Hombre";"Male";[@[Gender raw]])

Luego, ocultamos el campo "Genger raw.

A continuación, en el campo "Payment Method", decidimos escribir en mayúscula la primera letra de cada palabra como si fueran nombres propios. Para ello, cambiamos el nombre del campo a "Método de Pago Oculto" y creamos una nueva columna llamada "Método de Pago". Usamos la siguiente función:

In [ ]:
=NOMPROPIO([@[Payment Method raw]])

Finalmente, para los campos Customer Segment, Prefered Contact Method y Applied Discount, también queríamos asegurarnos de que todas las categorías de valor estuvieran escritas en inglés.En vez de aplicar el mismo método que en en la columna "Gender", creamos una tabla auxiliar en una hoja aparte. La tabla constaba de dos columnas: una para el término en español (Origen) y otra para el mismo término en inglés (Resultado).

| Origen     | Resultado |
|------------|-----------|
| Alto       | High      |
| Medio      | Medium    |
| Bajo       | Low       |
| Teléfono   | Phone     |
| VERDADERO  | Yes       |
| FALSO      | No        |

Creamos 3 nuevas columnas de estos campos y aplicamos las siguientes fórmulas, así conseguimos sustituir términos españoles por términos ingleses:

In [ ]:
=BUSCARV([@[Customer Segment raw]];Tablaauxiliar;2;FALSO)

=SI.ERROR(BUSCARV([@[Preferred Contact Method raw]];Tablaauxiliar;2;FALSO);[@[Preferred Contact Method raw]])

=BUSCARV([@[Applied Discount raw]];Tablaauxiliar;2;FALSO)

Finalmente, verificamos nuevamente si había filas duplicadas. Es común que, tras la limpieza y las transformaciones de datos, aparezcan filas duplicadas adicionales. Se eliminan 317 filas duplicadas.